Conectar Python a Cassandra en Docker

In [1]:
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'], port=9042)

session = cluster.connect()

session.execute("""
CREATE KEYSPACE IF NOT EXISTS test_keyspace 
WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

session.execute("USE test_keyspace")

session.execute("""
CREATE TABLE IF NOT EXISTS dataset_falso (
    nombre text,
    direccion text,
    correo text PRIMARY KEY,
    telefono text,
    fecha_nacimiento date,
    empresa text,
    numero_tarjeta text,
    fecha_vencimiento text,
    codigo_postal text
)
""")


Carga de Datos en la Tabla

In [2]:
import csv

with open('dataset_falso.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        session.execute("""
        INSERT INTO dataset_falso (nombre, direccion, correo, telefono, fecha_nacimiento, empresa, numero_tarjeta, fecha_vencimiento, codigo_postal)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['Nombre'],
            row['Dirección'],
            row['Correo'],
            row['Teléfono'],
            row['Fecha de nacimiento'],
            row['Empresa'],
            row['Número de tarjeta'],
            row['Fecha de vencimiento'],
            row['Código postal']
        ))

Verificación de los Datos Cargados

In [6]:
rows = session.execute("SELECT * FROM dataset_falso LIMIT 10")
for row in rows:
    print(row)

Row(correo='zbonilla@example.net', codigo_postal='41159', direccion='Alameda Dimas Esteve 83\nToledo, 39639', empresa='Rosalina Gaya Bolaños S.L.N.E', fecha_nacimiento=Date(-10946), fecha_vencimiento='11/29', nombre='Bruno del Bou', numero_tarjeta='4910858994417154', telefono='+34948567661')
Row(correo='emmaboix@example.org', codigo_postal='31890', direccion='Vial de Calixto Barreda 54\nBaleares, 31716', empresa='Hermanos Moliner S.A.', fecha_nacimiento=Date(-2054), fecha_vencimiento='03/30', nombre='Abril Lago Valle', numero_tarjeta='4712966667600830', telefono='+34 876 028 439')
Row(correo='inmaculada49@example.com', codigo_postal='42278', direccion='Cuesta Sandra Pera 218 Piso 1 \nCuenca, 30446', empresa='Distribuciones Españolas S.C.P', fecha_nacimiento=Date(8410), fecha_vencimiento='08/30', nombre='Leandro Araujo Peiró', numero_tarjeta='213117298068656', telefono='+34825438889')
Row(correo='montanajose-antonio@example.com', codigo_postal='07545', direccion='Camino de Rico Pera 4 P

Consultas de ejemplo

In [ ]:

# 1. Consultar todas las filas de la tabla
print("Consulta: Todas las filas")
rows = session.execute("SELECT * FROM dataset_falso LIMIT 5")
for row in rows:
    print(row)

# 2. Consultar una fila específica por el correo (clave primaria)
correo = 'ejemplo@correo.com'
print("\nConsulta: Fila específica por correo")
rows = session.execute("SELECT * FROM dataset_falso WHERE correo = %s", (correo,))
for row in rows:
    print(row)

# 3. Consultar los nombres y correos de los usuarios mayores de 50 años
print("\nConsulta: Nombres y correos de usuarios mayores de 50 años")
rows = session.execute("SELECT nombre, correo FROM dataset_falso WHERE fecha_nacimiento < '1973-01-01' ALLOW FILTERING")
for row in rows:
    print(row)

# 4. Contar el número total de registros en la tabla
print("\nConsulta: Contar número total de registros")
rows = session.execute("SELECT COUNT(*) FROM dataset_falso")
for row in rows:
    print(f"Total de registros: {row[0]}")

# 5. Consulta de empresas únicas en la tabla
print("\nConsulta: Empresas únicas")
rows = session.execute("SELECT empresa FROM dataset_falso GROUP BY empresa")
for row in rows:
    print(row.empresa)

# Cerrar la conexión
cluster.shutdown()

Consulta: Todas las filas
Row(correo='zbonilla@example.net', codigo_postal='41159', direccion='Alameda Dimas Esteve 83\nToledo, 39639', empresa='Rosalina Gaya Bolaños S.L.N.E', fecha_nacimiento=Date(-10946), fecha_vencimiento='11/29', nombre='Bruno del Bou', numero_tarjeta='4910858994417154', telefono='+34948567661')
Row(correo='emmaboix@example.org', codigo_postal='31890', direccion='Vial de Calixto Barreda 54\nBaleares, 31716', empresa='Hermanos Moliner S.A.', fecha_nacimiento=Date(-2054), fecha_vencimiento='03/30', nombre='Abril Lago Valle', numero_tarjeta='4712966667600830', telefono='+34 876 028 439')
Row(correo='inmaculada49@example.com', codigo_postal='42278', direccion='Cuesta Sandra Pera 218 Piso 1 \nCuenca, 30446', empresa='Distribuciones Españolas S.C.P', fecha_nacimiento=Date(8410), fecha_vencimiento='08/30', nombre='Leandro Araujo Peiró', numero_tarjeta='213117298068656', telefono='+34825438889')
Row(correo='montanajose-antonio@example.com', codigo_postal='07545', direccio

InvalidRequest: Error from server: code=2200 [Invalid query] message="SELECT DISTINCT queries must only request partition key columns and/or static columns (not empresa)"